In [14]:
import os 
import pandas as pd
from pathlib import Path

from survey_util import load_raw_survey_data, count_by_school_type, calculate_percentage, drop_duplicates

In [15]:
# Define paths and base directories

DATA_DIR = Path('../../data')
SURVEY_DIR = DATA_DIR / 'survey'
RAW_DIR = SURVEY_DIR / 'raw'

VIZ_DIR = Path('../../src/_data/viz/survey/phase_2')

raw_data = load_raw_survey_data()
base_respondents = len(raw_data)

SCHOOLS_DATA = pd.read_csv(Path(DATA_DIR, 'leeds_schools_public.csv'))

dfe_data = pd.read_csv(Path(DATA_DIR, 'spc_ees_school_characteristics.csv'), usecols={
    'time_period', 
    'la_name',
    'school_urn',
    'school_name',
    'phase_type_grouping',
    'typeofestablishment_name',
    'ward_code',
    'ward_name',
    'school_postcode',
    'school_size'
})


In [16]:
headline_questions = [
    '05_overall_rating_arts_offer', #excellent good
    '06_pupil_arts_entitlement', # yes no 
    '07_additional_arts_funding', #yesno
    '10_signposting_beyond_curriculum', #yesno
    # '13_arts_facilities', #list of facilities
    '14_rating_extra_curricular_arts_enrichment_offering', #excellent good
    '16_arts_award', #yesno
    '18_external_orgs_arts_performances', #yesno
    # '18a_arts_performance_external_org_subject_areas', #list of subjects
    '24_partnerships_external_orgs', #yesno
]


In [17]:
# Handle duplicates
raw_data_deduped = drop_duplicates(raw_data)

Headline stats

In [18]:

# Calculate values for headlines, out of total survey responses
excellent_arts_curriculum = int(((raw_data['05_overall_rating_arts_offer'].value_counts().Excellent) / base_respondents * 100))
pupil_arts_entitlement = int((len(raw_data.loc[raw_data['06_pupil_arts_entitlement']=='Yes']) / base_respondents * 100))
additional_arts_funding = int((len(raw_data.loc[raw_data['07_additional_arts_funding']=='Yes']) / base_respondents * 100))
signposting_beyond_curriculum = int((len(raw_data.loc[raw_data['10_signposting_beyond_curriculum']=='Yes']) / base_respondents * 100))
excellent_extra_curricular_arts = int(((raw_data['14_rating_extra_curricular_arts_enrichment_offering'].value_counts().Excellent) / base_respondents * 100))
arts_award = int((len(raw_data.loc[raw_data['16_arts_award']=='Yes']) / base_respondents * 100))
external_orgs_arts_performances = int((len(raw_data.loc[raw_data['18_external_orgs_arts_performances']=='Yes']) / base_respondents * 100))
partnerships_external_orgs = int((len(raw_data.loc[raw_data['24_partnerships_external_orgs']=='Yes']) / base_respondents * 100))

headlines_data = {
    'Title': [
        "Arts curriculum",
        "Arts entitlement",
        "Additional arts funding",
        "Signposting",
        "Extra-curricular offering",
        "Arts award",
        "External practitioners",
        "Local partnerships",
    ],
    'Value': [
        excellent_arts_curriculum,
        pupil_arts_entitlement,
        additional_arts_funding,
        signposting_beyond_curriculum,
        excellent_extra_curricular_arts,
        arts_award,
        external_orgs_arts_performances,
        partnerships_external_orgs
    ],
    'Note': [
        "Rated their arts curriculum as Excellent",
        "Said they have a commitment to pupil entitlement in the arts",
        "Said they seek additional funding for arts curriculum and enrichment",
        "Said they sign-post additional creative opportunities for students",
        "Rated their extra-curricular arts offering as Excellent",
        "Said their school offers an Arts Award",
        "Said they invited external creative practitioners into the school",
        "Said their school has a partnership with local arts organisations"
    ],
    'Suffix': '%'
}

headlines_index = headline_questions[:8]

headlines = pd.DataFrame(headlines_data, index=pd.Index(headlines_index, name='Question'))

headlines.to_csv(os.path.join(VIZ_DIR, 'headlines.csv'))


## Overview of Respondents

Survey responses by school type

In [19]:
# Responses broken down by school type

by_school_type = pd.DataFrame({
    'total_respondents' : raw_data_deduped.groupby('03_school_type')['03_school_type'].count(),
    'percent_respondents': raw_data_deduped.groupby('03_school_type')['03_school_type'].count().div(base_respondents).mul(100).round(0).astype(int),
    'unit': '%',
    'notes': [
        'of survey respondents were from a Primary school',
        'of survey respondents were from a Secondary school',
        'of survey respondents were from a Special school',
        'of survey respondents were from a Through school'
    ]
}).to_csv(os.path.join(VIZ_DIR, 'by_school_type.csv'))

## Arts Provision in Schools

Commitment to the arts

In [20]:
# 05_overall_rating_arts_offer - deduped data
overall_rating_arts_offer = (
    pd.read_csv(Path(VIZ_DIR, '05_overall_rating_arts_offer.csv'))
    .pivot_table(columns='School Type')
    .reset_index()
    .rename(columns={'index': 'Rating'})
)

overall_rating_arts_offer['Suffix'] = '%'
overall_rating_arts_offer.to_csv(Path(VIZ_DIR, 'overall_rating_arts_offer.csv'), index=False)


C:\Users\oi-user\AppData\Local\Temp\ipykernel_21392\2966034304.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  pd.read_csv(Path(VIZ_DIR, '05_overall_rating_arts_offer.csv'))


In [21]:
# 14_rating_extra_curricular_arts_enrichment_offering - deduped data

rating_extra_curricular_arts_enrichment_offering = (
    pd.read_csv(Path(VIZ_DIR, '14_rating_extra_curricular_arts_enrichment_offering.csv'))
    .pivot_table(columns='School Type')
    .reset_index()
    .rename(columns={'index': 'Rating'})
)

rating_extra_curricular_arts_enrichment_offering['Suffix'] = '%'
rating_extra_curricular_arts_enrichment_offering.to_csv(Path(VIZ_DIR, 'rating_extra_curricular_arts_enrichment_offering.csv'), index=False)


C:\Users\oi-user\AppData\Local\Temp\ipykernel_21392\1043124538.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  pd.read_csv(Path(VIZ_DIR, '14_rating_extra_curricular_arts_enrichment_offering.csv'))


Signposting headlines

In [22]:
# Signposting headlines - percentage of total survey responses

signposting = raw_data.loc[raw_data['10_signposting_beyond_curriculum']=='Yes']
competitions = raw_data.loc[raw_data['09_competitions_showcases']=='Yes']

responses = pd.DataFrame({
    'Competitions & Showcases' : [int(len(competitions)/base_respondents*100)],
    'Signposting beyond the curriculum': [int(len(signposting)/base_respondents*100)]
}).T.reset_index()

responses.columns = ['Headlines', 'Values']

Note = [
        'of all respondents said their school actively sign-posts for pupils to develop creative skills and participate in arts opportunities beyond the curriculum',
        'of all respondents said their school enters pupils into competitions or showcases - whether that is at a local community or regional/national level'
    ]

signposting_headlines = pd.DataFrame({
    'Headlines': responses['Headlines'],
    'Value': responses['Values'],
    'Suffix': '%',
    'Note': Note
}).to_csv(Path(VIZ_DIR, 'signposting_headlines.csv'), index=False)


External practitioners

In [23]:
partnerships_external_orgs = raw_data.loc[raw_data['24_partnerships_external_orgs']=='Yes']
external_orgs_arts_performances = raw_data.loc[raw_data['18_external_orgs_arts_performances']=='Yes']
partnerships_local_schools = raw_data.loc[raw_data['25_partnerships_local_schools']=='Yes']

responses = pd.DataFrame({
    'Partnerships' : [int(len(partnerships_external_orgs)/base_respondents*100)],
    'Delivering activities': [int(len(external_orgs_arts_performances)/base_respondents*100)],
    'School partnerships': [int(len(partnerships_local_schools)/base_respondents*100)]
}).T.reset_index()

responses.columns = ['Headlines', 'Values']

Note = [
        'of all respondents said their school has a partnership with any local arts organisations',
        'of all respondents said their school invited external practitioners to deliver creative activities',
        'of all respondents said their school partners with other schools to support the arts curriculum'
    ]

signposting_headlines = pd.DataFrame({
    'Headlines': responses['Headlines'],
    'Value': responses['Values'],
    'Suffix': '%',
    'Note': Note
}).to_csv(Path(VIZ_DIR, 'external_partnerships_headlines.csv'), index=False)

Subject areas of the organisations and creative practitioners engaging with schools:

In [24]:
# Subject areas of external practitioners - deduped data

subject_areas = raw_data_deduped[['02_school_name', '03_school_type', '18a_arts_performance_external_org_subject_areas']].dropna().reset_index()

subject_areas['18a_arts_performance_external_org_subject_areas'] = subject_areas['18a_arts_performance_external_org_subject_areas'].str.split(',')
subject_areas = subject_areas.explode('18a_arts_performance_external_org_subject_areas').drop(columns={'unique_response_number'})

subject_areas = subject_areas.groupby(['03_school_type', '18a_arts_performance_external_org_subject_areas']).count().reset_index()

subject_areas = subject_areas.rename(columns={
    '03_school_type': 'School Type',
    '18a_arts_performance_external_org_subject_areas': 'Subject Area',
    '02_school_name': 'Count'
})

counts_by_school_type = count_by_school_type(raw_data_deduped)

subject_areas['Percent'] = subject_areas.apply(lambda row: calculate_percentage(row, counts_by_school_type), axis=1)

subject_areas = subject_areas.pivot_table(columns=['School Type'], index=['Subject Area'], values=['Percent']).reset_index().set_index('Subject Area')

subject_areas.columns = ['Primary', 'Secondary', 'Special School', 'Through School']

subject_areas = subject_areas.fillna(0).astype(int)

subject_areas.to_csv(Path(VIZ_DIR, 'subject_areas_headlines.csv'), index=True)

subject_areas


,Primary,Secondary,Special School,Through School
Subject Area,,,,
Art,29,33,33,50
Dance,31,67,17,50
Design and Technology,14,17,0,0
Digital media and gaming,3,33,0,50
Film,3,0,0,0
Museum/Heritage,17,50,17,50
Music,51,67,50,100
Theatre,40,67,83,50


Community arts programmes:

In [25]:
# Community programs - total survey responses

community_based_arts_programmes = raw_data.loc[raw_data['26_community_based_arts_programmes']=='Yes']
national_arts_programmes = raw_data.loc[raw_data['23_national_programmes']=='Yes']

responses = pd.DataFrame({
    'Community-based arts programmes' : [int(len(community_based_arts_programmes)/base_respondents*100)],
    'National arts programmes' : [int(len(national_arts_programmes)/base_respondents*100)],
}).T.reset_index()

responses.columns = ['Headlines', 'Values']

Note = [
        'of all respondents said their school is part of a local/community-based arts programme.',
        'of all respondents said their school is part of a national arts programme.',
    ]

signposting_headlines = pd.DataFrame({
    'Headlines': responses['Headlines'],
    'Value': responses['Values'],
    'Suffix': '%',
    'Note': Note
}).to_csv(Path(VIZ_DIR, 'community_programs_headlines.csv'), index=False)

## Facilities

In [ ]:
# Facilities - deduped data

facilities = raw_data_deduped[['02_school_name', '03_school_type', '13_arts_facilities']].dropna().reset_index()

facilities['13_arts_facilities'] = facilities['13_arts_facilities'].str.split(',')
facilities = facilities.explode('13_arts_facilities').drop(columns={'unique_response_number'})

facilities = facilities.groupby(['03_school_type', '13_arts_facilities']).count().reset_index()

facilities = facilities.rename(columns={
    '03_school_type': 'School Type',
    '13_arts_facilities': 'Facilities',
    '02_school_name': 'Count'
})

counts_by_school_type = count_by_school_type(raw_data_deduped)

facilities['Percent'] = facilities.apply(lambda row: calculate_percentage(row, counts_by_school_type), axis=1)

facilities = facilities.pivot_table(columns=['School Type'], index=['Facilities'], values=['Percent']).reset_index().set_index('Facilities')

facilities.columns = ['Primary', 'Secondary', 'Special School', 'Through School']

facilities = facilities.fillna(0).astype(int)

facilities.to_csv(Path(VIZ_DIR, 'facilities_headlines.csv'), index=True)

facilities

In [44]:
# Facilities - deduped data

facilities = raw_data_deduped[['02_school_name', '03_school_type', '13_arts_facilities']].dropna().reset_index()


facilities['13_arts_facilities'] = facilities['13_arts_facilities'].str.split(',')
facilities = facilities.explode('13_arts_facilities').drop(columns={'unique_response_number'})

facilities = facilities.groupby(['03_school_type', '13_arts_facilities']).size().reset_index(name='Count')

counts_by_school_type = count_by_school_type(raw_data_deduped)

facilities['Total'] = facilities.groupby('03_school_type')['Count'].transform('sum').round(0)

facilities['Percent'] = ((facilities['Count'] / facilities['Total']) * 100).round(0).astype(int)

facilities_pivot = facilities.pivot_table(index='13_arts_facilities', columns='03_school_type', values='Percent', fill_value=0)

# facilities_pivot['Total'] = facilities_pivot.sum(axis=1).round(0).astype(int)

facilities_pivot.reset_index(inplace=True)

facilities_pivot.columns.name = None 
facilities_pivot.rename(columns={'Primary': 'Primary', 'Secondary': 'Secondary', 'Special': 'Special School', 'Through': 'Through School'}, inplace=True)

facilities_pivot.to_csv(Path(VIZ_DIR, 'facilities.csv'), index=False)


## Funding & Support

Funding sources for provision of art curriculum and enrichment activity: